# Учебный Проект → «Прогнозирование оттока клиентов»

<div class="alert alert-info" role="alert" style="border-radius: 7px">
    <span style="font-size:0.7rem">Student's note 00</span>
    <p class="mb-0"><strong>Комментарий:</strong></p>
    <p>Арсен, здравствуй.</p>
    <p>Спасибо за <code>вводную консультацию</code>.</p>
</div>

## Телеком — задача проекта

>Оператор связи «Ниединогоразрыва.ком» хочет научиться прогнозировать отток клиентов. Если выяснится, что пользователь планирует уйти, ему будут предложены промокоды и специальные условия. Команда оператора собрала персональные данные о некоторых клиентах, информацию об их тарифах и договорах.

✅ название и описание проекта, цель проекта

In [1]:
import pandas as pd
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

**Настроим вид графиков по+красоте** ✨

In [2]:
%config InlineBackend.figure_formats = ['svg']

In [3]:
# style MATPLOTLIBRC
custom_params = {
                'figure.figsize': (10, 6),
                'figure.facecolor': '#232425',
                'figure.dpi': 240,

                'legend.frameon': False,
                'legend.borderpad': 1.4,
                'legend.labelspacing': 0.7,
                'legend.handlelength': 0.7,
                'legend.handleheight': 0.7,

                'axes.facecolor': '#232425',
                'axes.labelcolor': '#EEEEEE',
                'axes.labelpad': 17,
                'axes.spines.left': False,
                'axes.spines.bottom': False,
                'axes.spines.right': False,
                'axes.spines.top': False,
                'axes.grid': False,

                'contour.linewidth': 0.0,

                'xtick.color': '#AAAAAA',
                'ytick.color': '#AAAAAA',
                'xtick.bottom': True,
                'xtick.top': False,
                'ytick.left': True,
                'ytick.right': False,
    
                "lines.color": '#EEEEEE',

                'text.color': '#EEEEEE',
    
                'font.family': 'sans-serif',
            }

In [4]:
# set max columns to none
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)

**Константы:**

In [5]:
RANDOM_SEED = 270223
DPI_K = custom_params['figure.dpi'] / mpl.rcParams['figure.dpi']
PX = 1/custom_params['figure.dpi']
PY_CASE = re.compile(r'(?<!^)(?<![A-Z])(?=[A-Z])')
TEST_SIZE = 0.25
DATA_UPDATE = np.datetime64('2020-02-01')

**Функции:**

In [6]:
def columns_w_na(data_df):
    col_nan = data_df.columns[
            data_df.isna().any()
        ].tolist()
    df_length = len(data_df.index)

    if len(col_nan) != 0:
        print('Колонки с NaN:')
        for col in col_nan:
            count_na = len(
                    data_df.loc[data_df[col].isna()].index
                )

            print(
                '{: .2%}'.format(count_na/df_length),
                end='\t→ '
            )
            print(col, end=' → ')
            print(count_na)
    else:
        print('Пропусков NaN в наборе данных нет.\n')

In [7]:
def baisic_df_info(data_df, title='Базовая информация'):
    print(title,  '→ о наборе данных:', end='\n\n')
    print('Дубликатов:',
             len(data_df.loc[data_df.duplicated()].index),
          end='\n\n'
     )
    
    columns_w_na(data_df)
    
    data_df.info()
    
    print()
    display(
        data_df.sample(5),
        data_df.describe(),
    )

In [8]:
def to_snake_case(df, pattern=PY_CASE):
    to_return  = pd.Series(df.columns).apply(
            lambda c: re.sub(pattern, '_',  c ).lower()
        )
    
    return to_return

In [9]:
def to_px(size_px):
    px_density = int(
        round(
            size_px * PX * DPI_K
        )
    )
    return px_density

In [10]:
def plot_hist(data, title='Histogram', custom_params=custom_params):
    
    num_col = 2
    num_row = int(
            len(data.columns) / num_col
        ) + (len(data.columns) % num_col)
    
    with mpl.rc_context(custom_params):
        fig, axs = plt.subplots(
            nrows=num_row,
            ncols=num_col,
            figsize=(
                to_px(960), to_px(960)),
            tight_layout=True
        )
        
        for n, col_name in enumerate(data.columns):
            ax_row = int(n/2)
            ax_col = n%2
            n_bins = len(data[col_name].unique())
                
            axs[ax_row, ax_col].hist(
                data[col_name],
                bins=n_bins,
                color='forestgreen'
            )
            axs[ax_row, ax_col].set_title(col_name)
        
        fig.suptitle(title, fontsize=16)
        
        if len(data.columns) % num_col:
            fig.delaxes(axs[-1, -1])

        plt.show()

**Загружаем наборы данных**

In [11]:
try:
    contract_df = pd.read_csv(
            './datasets/contract.csv',
        )
    internet_df = pd.read_csv(
            './datasets/internet.csv',
        )
    personal_df = pd.read_csv(
            './datasets/personal.csv',
        )
    phone_df = pd.read_csv(
            './datasets/phone.csv',
        )
except FileNotFoundError:
    contract_df = pd.read_csv(
            '/datasets/final_provider/contract.csv',
        )
    internet_df = pd.read_csv(
            '/datasets/final_provider/internet.csv',
        )
    personal_df = pd.read_csv(
            '/datasets/final_provider/personal.csv',
        )
    phone_df = pd.read_csv(
            '/datasets/final_provider/phone.csv',
        )
    print('FYI datasets loaded via url')

Сделаем названия столбцов по+красоте → `snake_case`  
Во имя Python:  
<img src="https://s3.dualstack.us-east-2.amazonaws.com/pythondotorg-assets/media/files/python-logo-only.svg" alt="Python logo" width="27px"/>

In [12]:
contract_df.columns  = to_snake_case(contract_df)
internet_df.columns  = to_snake_case(internet_df)
personal_df.columns  = to_snake_case(personal_df)
phone_df.columns  = to_snake_case(phone_df)

In [13]:
baisic_df_info(contract_df, title='Контракты')

Контракты → о наборе данных:

Дубликатов: 0

Пропусков NaN в наборе данных нет.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7043 non-null   object 
 1   begin_date         7043 non-null   object 
 2   end_date           7043 non-null   object 
 3   type               7043 non-null   object 
 4   paperless_billing  7043 non-null   object 
 5   payment_method     7043 non-null   object 
 6   monthly_charges    7043 non-null   float64
 7   total_charges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB



,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges
5846,3370-HXOPH,2019-11-01,No,Month-to-month,No,Electronic check,76.10,257.6
737,2521-NPUZR,2014-04-01,No,Two year,No,Credit card (automatic),101.00,7085.5
6710,4342-HFXWS,2016-02-01,No,One year,No,Bank transfer (automatic),69.70,3023.65
4649,5172-RKOCB,2014-02-01,No,Two year,Yes,Credit card (automatic),108.95,7875
3117,9844-FELAJ,2014-04-01,No,One year,Yes,Electronic check,107.90,7475.85


,monthly_charges
count,7043.000000
mean,64.761692
std,30.090047
min,18.250000
25%,35.500000
50%,70.350000
75%,89.850000
max,118.750000


In [14]:
contract_df.loc[
        contract_df['end_date'] == 'No',
        'end_date'
    ] = pd.NaT

contract_df['end_date'] = pd.to_datetime(
        contract_df['end_date']
    )

contract_df['end_date'].dt.date

print(
    'Ушедших пользователей',
    contract_df.loc[contract_df['end_date'].notnull(), 'customer_id'].count()
)

Ушедших пользователей 1869


In [15]:
contract_df['begin_date'] = pd.to_datetime(
        contract_df['begin_date']
    )

In [16]:
display(
    contract_df.loc[
        contract_df['total_charges'] == ' '
    ]
)

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges
488,4472-LVYGI,2020-02-01,NaT,Two year,Yes,Bank transfer (automatic),52.55,
753,3115-CZMZD,2020-02-01,NaT,Two year,No,Mailed check,20.25,
936,5709-LVOEQ,2020-02-01,NaT,Two year,No,Mailed check,80.85,
1082,4367-NUYAO,2020-02-01,NaT,Two year,No,Mailed check,25.75,
1340,1371-DWPAZ,2020-02-01,NaT,Two year,No,Credit card (automatic),56.05,
3331,7644-OMVMY,2020-02-01,NaT,Two year,No,Mailed check,19.85,
3826,3213-VVOLG,2020-02-01,NaT,Two year,No,Mailed check,25.35,
4380,2520-SGTTA,2020-02-01,NaT,Two year,No,Mailed check,20.00,
5218,2923-ARZLG,2020-02-01,NaT,One year,Yes,Mailed check,19.70,
6670,4075-WKNIU,2020-02-01,NaT,Two year,No,Mailed check,73.35,


In [17]:
print(
    'Поля абонентская плата и всего заплачено совпадают',
    len(contract_df.loc[
            contract_df['monthly_charges'] == contract_df['total_charges']
        ].index),
    'раз'
)

Поля абонентская плата и всего заплачено совпадают 0 раз


Предположим что **«Ниединогоразрыва.ком»** использует **предоплатную** систему расчета.  
Тогда прировняем пустые `total_charges` к `monthly_charges`.

In [18]:
contract_df.loc[
            contract_df['total_charges'] == ' ',
            'total_charges'
    ] = contract_df.loc[
            contract_df['total_charges'] == ' ',
            'monthly_charges'
    ]

In [19]:
contract_df.loc[:, 
        ['monthly_charges', 'total_charges' ]
    ] = contract_df.loc[:, 
        ['monthly_charges', 'total_charges' ]].astype({
            'monthly_charges': 'float16',
             'total_charges': 'float32' 
        })

/tmp/ipykernel_12347/4100139373.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  contract_df.loc[:,


In [20]:
display(
    contract_df.sample(5)
)
contract_df.info()

,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges
499,7783-YKGDV,2018-10-01,2019-10-01,Month-to-month,Yes,Bank transfer (automatic),99.68750,1238.449951
5351,5649-VUKMC,2018-12-01,2019-12-01,One year,No,Mailed check,95.50000,1115.150024
4450,4925-LMHOK,2018-08-01,2019-11-01,Month-to-month,Yes,Credit card (automatic),58.59375,939.700012
6908,1750-CSKKM,2019-02-01,2019-11-01,Month-to-month,No,Electronic check,55.34375,449.750000
607,8672-OAUPW,2015-11-01,NaT,One year,No,Credit card (automatic),47.84375,2356.750000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        7043 non-null   object        
 1   begin_date         7043 non-null   datetime64[ns]
 2   end_date           1869 non-null   datetime64[ns]
 3   type               7043 non-null   object        
 4   paperless_billing  7043 non-null   object        
 5   payment_method     7043 non-null   object        
 6   monthly_charges    7043 non-null   float16       
 7   total_charges      7043 non-null   float32       
dtypes: datetime64[ns](2), float16(1), float32(1), object(4)
memory usage: 371.5+ KB


In [21]:
baisic_df_info(internet_df, 'Интернет')

Интернет → о наборе данных:

Дубликатов: 0

Пропусков NaN в наборе данных нет.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   customer_id        5517 non-null   object
 1   internet_service   5517 non-null   object
 2   online_security    5517 non-null   object
 3   online_backup      5517 non-null   object
 4   device_protection  5517 non-null   object
 5   tech_support       5517 non-null   object
 6   streaming_tv       5517 non-null   object
 7   streaming_movies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB



,customer_id,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies
1360,4116-TZAQJ,Fiber optic,Yes,No,No,No,No,No
3889,1862-SKORY,DSL,No,Yes,No,No,Yes,No
1906,1623-NLDOT,DSL,No,Yes,No,Yes,No,No
2488,7890-VYYWG,DSL,No,No,No,No,Yes,No
2654,4951-UKAAQ,Fiber optic,No,Yes,No,No,No,Yes


,customer_id,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies
count,5517,5517,5517,5517,5517,5517,5517,5517
unique,5517,2,2,2,2,2,2,2
top,7590-VHVEG,Fiber optic,No,No,No,No,No,No
freq,1,3096,3498,3088,3095,3473,2810,2785


In [22]:
baisic_df_info(personal_df, title='Пользователи')

Пользователи → о наборе данных:

Дубликатов: 0

Пропусков NaN в наборе данных нет.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     7043 non-null   object
 1   gender          7043 non-null   object
 2   senior_citizen  7043 non-null   int64 
 3   partner         7043 non-null   object
 4   dependents      7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB



,customer_id,gender,senior_citizen,partner,dependents
3840,1080-BWSYE,Male,1,Yes,No
4199,4088-YLDSU,Male,0,Yes,No
2520,5788-YPOEG,Female,0,Yes,Yes
3638,7579-OOPEC,Female,1,Yes,No
5062,2911-UREFD,Female,0,Yes,No


,senior_citizen
count,7043.000000
mean,0.162147
std,0.368612
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [23]:
baisic_df_info(phone_df, title='Телефония')

Телефония → о наборе данных:

Дубликатов: 0

Пропусков NaN в наборе данных нет.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     6361 non-null   object
 1   multiple_lines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB



,customer_id,multiple_lines
3796,2642-MAWLJ,No
1276,6960-HVYXR,Yes
4520,6527-PZFPV,No
4061,2665-NPTGL,Yes
971,6166-ILMNY,No


,customer_id,multiple_lines
count,6361,6361
unique,6361,2
top,5575-GNVDE,No
freq,1,3390


✅ все датафреймы должны быть исследованы (методы describe(), info(), можно добавить графики)

## План проекта

1. Анализируем наборы данных  
    1.1. Предобрабатываем данные
2. Выбираем необходимые `features`  
    2.1. Смотрим на корреляции  
    2.2. Включаем логику  
    2.3. Создаём признаки  
    2.4. Создаём `targets` из `'end_date'`  
    2.5. Объединяем наборы данных в один
3. Тестируем разные модели  
    3.0. Делим набор данных на тестовую и тренировочную выборки
    3.1. `RandomForestClassifier`  
    3.2. `LogisticRegression`  
    3.3. `CatBoostClassifier`  
4. Тестируем лучшую модель на кроссвалидации модель
5. Пишем выводы


✅ содержание (план) проекта: здесь нужно привести структуру проекта, т.е. основные пункты и подпункты. План должен быть достаточно подробным, обычном минимум 4-5 основных пунктов с подпунктами.

### Вопросы

1. Предоплатная ли система расчётов у **«Ниединогоразрыва.ком»?**
2. Может ли нейронная сеть быть классификатором, какая, что почитать, пожалуйста?
3. Какие действия = пункты плана можно еще выполнить?
4. Где раздобыть еще выборку по ушедшим абонентам; Могут ли **«Ниединогоразрыва.ком»** предоставить еще информацию?)
5. Есть ли ещё что+то важное что стоит учесть до начала действий, пожалуйста?

✅ вопросы и комментарии к тимлиду (если вопросов нет, нужно явно про это написать, иначе проект будет возвращен для получения возможных вопросов студента)

<hr>

## Основная часть проекта

to be continued…

<hr>

## Отчет

to be continued…

<hr>

>✍️ Основные тезисы вводной консультации:
> - фиксируем параметр RANDOM_STATE = 270223 (дата начала финального спринта);
> - дедлайн сдачи отчета 11 марта 23:59 по мск (вторая суббота);
> - размер тестовой выборки: test_size = 0.25 (строго!!!);
> - рекомендуется использовать OneHotEncoder для кодирования категориальных признаков. Кодируем после разделения на train и test, чтобы не было утечки в данных;
> - если при кодировании используется get_dummies, то нужно сравнить списки получившихся признаков (должно быть полное совпадение);
> - помимо основной части, в проекте нужно сделать визуализацию графиков, исследовать корреляцию признаков. Для числовых признаков используем корреляцию Спирмена, для категориальных - Крамера. Можно применять инструмент фи-корреляции для смешанного набора признаков (библиотека phik);
> - для линейных моделей нужно масштабировать числовые признаки;
> - про баланс классов: Upsampling использовать не нужно (!!!), лучше используйте балансировку классов внутри модели;
> - на выборке test проверяется только одна лучшая модель, а сравнение моделей делаем с использованием кросс-валидации.
  
© Арсен Абдулин